The distribution of inputs to deeper layers may vary after weigt update after every mini batch.  This can cause the learning algorithm to forever chase a moving target. (interal covariate shift)

BN standardizes input to layer for each mini batch, stabilizing the learning process and faster convergence

## Batch Norm
Step 1: zero center & Normalize input

Step 2: Scale & Shift using two new parameter

Steps:

$\mu = \frac{1}{n} \sum x $

$ \sigma ^2 = \frac{1}{n} \sum (x - \mu)^2 $

$ \hat x = \frac {x - \mu}{\sqrt{\sigma ^2 + \epsilon}} $

$ z = \gamma * \hat x + \beta $
Batch Norm is done in batch of training data, hence the name. the "n" in calculation represents the batch size

$\gamma$ -> output scale parametet, $\beta$ -> output offset/shift parameter

z -> rescaled and shifted version of inputs

Testing
When working with batches, the mean and sigma can vary lot

One solution : run the whole training set through the neural network and compute the mean and standard deviation of each input of the BN layer.

estimate these final statistics during training by using a moving average of layer’s input means and standard deviations.

Hence, four vectors are learned at each batch-norm layer:

$\gamma$ : output scale vector
$\beta$ : output offset vector
$\mu$ : final input mean vector (exp mov avg)
$\sigma$ : final input std dev vector (exp mov avg)
The $\mu, \sigma$ are estimated only at training and used only at testing


## Summary:

It had solved vanishing/exploding grads even with saturating activation functions (sigmoid and logistic)

Faster convergence but training is slow (bcos more computation)

In [ ]:
import tensorflow as tf


model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = [28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='elu'),
    tf.keras.layers.Dense(10, activation='softmax'),
    ])

In [ ]:
model.summary()

Notice the batchnorm has 4 parameters per input feature. Here the input is 784 features

Implies 4*784 = 3136

The $\mu, \sigma$ at bach norm layer is not used during back-prop, hence it is shown as non-trainable params.

ie 2*784 = 1568

In [ ]:
[(var.name, var.trainable) for var in model.layers[1].variables]

Where to add Batch Norm layer
The authors of the BN paper argued in favor of adding the BN layers before the activation functions (for Relu), rather than after. For sigmoid, tanh BN should be used for the activation. (a/c to machinelearningmastery)

Depends on our dataset - needs experimentation

To add the BN layers before the activation functions, you must remove the activation function from the hidden layers and add them as separate layers after the BN layers.
Moreover, since a Batch Normalization layer includes one offset parameter per input, you can remove the bias term from the previous layer (just pass use_bias=False when creating it)

BN provides some level of regularization, hence dont use with dropout. Dropout introduces randomness that can confuse BN by adding noise to the normalization.      

https://machinelearningmastery.com/batch-normalization-for-training-of-deep-neural-networks/
https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html (for back prop derivation )
https://github.com/wiseodd/hipsternet/blob/master/hipsternet/layer.py (code below)


In [ ]:
def bn_forward(X, gamma, beta, cache, momentum=.9, train=True):
    running_mean, running_var = cache

    if train:
        mu = np.mean(X, axis=0)
        var = np.var(X, axis=0)

        X_norm = (X - mu) / np.sqrt(var + c.eps)
        out = gamma * X_norm + beta

        cache = (X, X_norm, mu, var, gamma, beta)

        running_mean = util.exp_running_avg(running_mean, mu, momentum)
        running_var = util.exp_running_avg(running_var, var, momentum)
    else:
        X_norm = (X - running_mean) / np.sqrt(running_var + c.eps)
        out = gamma * X_norm + beta
        cache = None
        
    return out, cache, running_mean, running_var

https://blog.paperspace.com/busting-the-myths-about-batch-normalization/ 
BN gives us control on the output distribution of each layer. 


BN helps maintain the input to the linear region of the activation? So even sigmoid can be used in a better way? Doubt!